## 第7章: 単語ベクトル
単語の意味を実ベクトルで表現する単語ベクトル（単語埋め込み）に関して，以下の処理を行うプログラムを作成せよ．

#### 60. 単語ベクトルの読み込みと表示
Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

指定の学習済み単語ベクトルをダウンロードします。

In [1]:
%load_ext lab_black

In [2]:
# FILE_ID = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"
# FILE_NAME = "GoogleNews-vectors-negative300.bin.gz"
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt

# 下記からダウンロードできます。
# https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

自然言語処理のさまざまなタスクで利用されるGensimを用いて、単語ベクトルを読み込みます。

In [3]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(
    "./GoogleNews-vectors-negative300.bin.gz", binary=True
)

読み込んだ後は、ベクトル化したい単語を指定するだけで簡単に単語ベクトルを得ることができます。

In [4]:
model["United_States"]

array([-3.61328125e-02, -4.83398438e-02,  2.35351562e-01,  1.74804688e-01,
       -1.46484375e-01, -7.42187500e-02, -1.01562500e-01, -7.71484375e-02,
        1.09375000e-01, -5.71289062e-02, -1.48437500e-01, -6.00585938e-02,
        1.74804688e-01, -7.71484375e-02,  2.58789062e-02, -7.66601562e-02,
       -3.80859375e-02,  1.35742188e-01,  3.75976562e-02, -4.19921875e-02,
       -3.56445312e-02,  5.34667969e-02,  3.68118286e-04, -1.66992188e-01,
       -1.17187500e-01,  1.41601562e-01, -1.69921875e-01, -6.49414062e-02,
       -1.66992188e-01,  1.00585938e-01,  1.15722656e-01, -2.18750000e-01,
       -9.86328125e-02, -2.56347656e-02,  1.23046875e-01, -3.54003906e-02,
       -1.58203125e-01, -1.60156250e-01,  2.94189453e-02,  8.15429688e-02,
        6.88476562e-02,  1.87500000e-01,  6.49414062e-02,  1.15234375e-01,
       -2.27050781e-02,  3.32031250e-01, -3.27148438e-02,  1.77734375e-01,
       -2.08007812e-01,  4.54101562e-02, -1.23901367e-02,  1.19628906e-01,
        7.44628906e-03, -

#### 61. 単語の類似度
“United States”と”U.S.”のコサイン類似度を計算せよ．

ここではsimilarityメソッドを利用します。2単語を指定すると、単語間のコサイン類似度を計算することができます。

In [5]:
model.similarity("United_States", "U.S.")

0.73107743

#### 62. 類似度の高い単語10件
“United States”とコサイン類似度が高い10語と，その類似度を出力せよ．

ここではmost_similarメソッドを利用します。単語を指定すると、topnまでの類似度上位単語とその類似度を取得することができます。



In [6]:
model.most_similar("United_States", topn=10)

[('Unites_States', 0.7877248525619507),
 ('Untied_States', 0.7541370987892151),
 ('United_Sates', 0.7400724291801453),
 ('U.S.', 0.7310774326324463),
 ('theUnited_States', 0.6404393911361694),
 ('America', 0.6178410053253174),
 ('UnitedStates', 0.6167312264442444),
 ('Europe', 0.6132988929748535),
 ('countries', 0.6044804453849792),
 ('Canada', 0.601906955242157)]

####　63. 加法構成性によるアナロジー
“Spain”の単語ベクトルから”Madrid”のベクトルを引き，”Athens”のベクトルを足したベクトルを計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

前問でも利用したmost_similarメソッドは、足すベクトルと引くベクトルをそれぞれ指定した上で、計算後のベクトルと類似度が高い単語を取得することができます。
ここでは、問題文の指示に従い、Spain - Madrid + Athensのベクトルと類似度の高い単語を表示していますが、期待通りGreeceが1位に登場しています。

In [7]:
vec = model["Spain"] - model["madrid"] + model["Athens"]
model.most_similar(positive=["Spain", "Athens"], negative=["Madrid"], topn=10)

[('Greece', 0.6898480653762817),
 ('Aristeidis_Grigoriadis', 0.560684859752655),
 ('Ioannis_Drymonakos', 0.5552908778190613),
 ('Greeks', 0.545068621635437),
 ('Ioannis_Christou', 0.5400862097740173),
 ('Hrysopiyi_Devetzi', 0.5248445272445679),
 ('Heraklio', 0.5207759737968445),
 ('Athens_Greece', 0.516880989074707),
 ('Lithuania', 0.5166865587234497),
 ('Iraklion', 0.5146791338920593)]

#### 64. アナロジーデータでの実験
単語アナロジーの評価データをダウンロードし，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．

指定のデータをダウンロードします。

In [8]:
!wget http://download.tensorflow.org/data/questions-words.txt

--2021-12-12 11:30:41--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.42.176, 2404:6800:4004:810::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.42.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K   457KB/s    in 1.3s    

2021-12-12 11:30:44 (457 KB/s) - ‘questions-words.txt’ saved [603955/603955]



In [9]:
# 先頭10行の確認
!head -10 questions-words.txt

: capital-common-countries
Athens Greece Baghdad Iraq
Athens Greece Bangkok Thailand
Athens Greece Beijing China
Athens Greece Berlin Germany
Athens Greece Bern Switzerland
Athens Greece Cairo Egypt
Athens Greece Canberra Australia
Athens Greece Hanoi Vietnam
Athens Greece Havana Cuba


1行ずつ読込み、指定の単語と類似度を計算した上で整形したデータを出力します。

In [10]:
with open("./questions-words.txt", "r") as f1, open(
    "./questions-words-add.txt", "w"
) as f2:
    for line in f1:  # f1から1行ずつ読込み、求めた単語と類似度を追加してf2に書込む
        line = line.split()
        if line[0] == ":":
            category = line[1]
        else:
            word, cos = model.most_similar(
                positive=[line[1], line[2]], negative=[line[0]], topn=1
            )[0]
            f2.write(" ".join([category] + line + [word, str(cos) + "\n"]))

In [11]:
!head -10 questions-words-add.txt

capital-common-countries Athens Greece Baghdad Iraq Iraqi 0.635187029838562
capital-common-countries Athens Greece Bangkok Thailand Thailand 0.7137669324874878
capital-common-countries Athens Greece Beijing China China 0.7235778570175171
capital-common-countries Athens Greece Berlin Germany Germany 0.6734622716903687
capital-common-countries Athens Greece Bern Switzerland Switzerland 0.4919748306274414
capital-common-countries Athens Greece Cairo Egypt Egypt 0.7527808547019958
capital-common-countries Athens Greece Canberra Australia Australia 0.583732545375824
capital-common-countries Athens Greece Hanoi Vietnam Viet_Nam 0.6276341676712036
capital-common-countries Athens Greece Havana Cuba Cuba 0.6460990905761719
capital-common-countries Athens Greece Helsinki Finland Finland 0.68999844789505


#### 65. アナロジータスクでの正解率
64の実行結果を用い，意味的アナロジー（semantic analogy）と文法的アナロジー（syntactic analogy）の正解率を測定せよ．

対応するカテゴリごとにそれぞれ計算します。

In [12]:
with open("./questions-words-add.txt", "r") as f:
    sem_cnt = 0
    sem_cor = 0
    syn_cnt = 0
    syn_cor = 0
    for line in f:
        line = line.split()
        if not line[0].startswith("gram"):
            sem_cnt += 1
            if line[4] == line[5]:
                sem_cor += 1
        else:
            syn_cnt += 1
            if line[4] == line[5]:
                syn_cor += 1

print(f"意味的アナロジー正解率: {sem_cor/sem_cnt:.3f}")
print(f"文法的アナロジー正解率: {syn_cor/syn_cnt:.3f}")

意味的アナロジー正解率: 0.731
文法的アナロジー正解率: 0.740


#### 66. WordSimilarity-353での評価
The WordSimilarity-353 Test Collectionの評価データをダウンロードし，単語ベクトルにより計算される類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

このデータは、単語のペアに対して人間が評価した類似度が付与されています。
それぞれのペアに対して単語ベクトルの類似度を計算し、両者のスピアマン順位相関係数を計算します。

In [13]:
!wget http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
!unzip wordsim353.zip

--2021-12-12 13:09:34--  http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving www.gabrilovich.com (www.gabrilovich.com)... 208.97.177.37
Connecting to www.gabrilovich.com (www.gabrilovich.com)|208.97.177.37|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gabrilovich.com/resources/data/wordsim353/wordsim353.zip [following]
--2021-12-12 13:09:35--  https://gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving gabrilovich.com (gabrilovich.com)... 208.97.177.37
Connecting to gabrilovich.com (gabrilovich.com)|208.97.177.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257 (23K) [application/zip]
Saving to: ‘wordsim353.zip’

wordsim353.zip      100%[===================>]  22.71K   133KB/s    in 0.2s    

2021-12-12 13:09:37 (133 KB/s) - ‘wordsim353.zip’ saved [23257/23257]

Archive:  wordsim353.zip
  inflating: combined.csv            
  inflating: set1.csv                
  

In [14]:
!head -10 './combined.csv'

Word 1,Word 2,Human (mean)
love,sex,6.77
tiger,cat,7.35
tiger,tiger,10.00
book,paper,7.46
computer,keyboard,7.62
computer,internet,7.58
plane,car,5.77
train,car,6.31
telephone,communication,7.50


In [15]:
# 国名の取得
countries = set()
with open("./questions-words-add.txt") as f:
    for line in f:
        line = line.split()
        if line[0] in ["capital-common-countries", "capital-world"]:
            countries.add(line[2])
        elif line[0] in ["currency", "gram6-nationality-adjective"]:
            countries.add(line[1])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

#### 67. k-meansクラスタリング
国名に関する単語ベクトルを抽出し，k-meansクラスタリングをクラスタ数k=5として実行せよ．

適当な国名リストの取得元が見つからなかったため、単語アナロジーの評価データから収集しています。

In [16]:
# 国名の取得
countries = set()
with open("./questions-words-add.txt") as f:
    for line in f:
        line = line.split()
        if line[0] in ["capital-common-countries", "capital-world"]:
            countries.add(line[2])
        elif line[0] in ["currency", "gram6-nationality-adjective"]:
            countries.add(line[1])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

In [18]:
import numpy as np
from sklearn.cluster import KMeans

# k-meansクラスタリング
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print("cluster", i)
    print(", ".join([countries[k] for k in cluster]))

cluster 0
Nigeria, Ghana, Kenya, Malawi, Mauritania, Burundi, Niger, Mozambique, Gambia, Angola, Liberia, Zimbabwe, Sudan, Madagascar, Senegal, Botswana, Guinea, Mali, Zambia, Eritrea, Somalia, Gabon, Rwanda, Uganda, Libya, Algeria, Tunisia, Namibia
cluster 1
Norway, Liechtenstein, Greenland, Denmark, Iceland, Canada, Switzerland, Austria, Sweden, Finland, Germany, Belgium, Netherlands
cluster 2
Uzbekistan, Ukraine, Montenegro, Poland, Macedonia, Croatia, Belarus, Bulgaria, Albania, Russia, Latvia, Lithuania, Azerbaijan, Armenia, Slovakia, Turkmenistan, Cyprus, Kazakhstan, Slovenia, Estonia, Romania, Serbia, Kyrgyzstan, Moldova, Hungary, Georgia, Tajikistan
cluster 3
Syria, Taiwan, Bangladesh, Thailand, Portugal, Malaysia, Australia, Turkey, Qatar, France, Pakistan, Ireland, England, Indonesia, Israel, Oman, Malta, Lebanon, India, Europe, Iran, Spain, Afghanistan, Bahrain, China, USA, Iraq, Korea, Japan, Vietnam, Morocco, Italy, Egypt, Jordan, Greece
cluster 4
Cambodia, Bahamas, Domini

#### 68. Ward法によるクラスタリング
国名に関する単語ベクトルに対し，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

In [19]:
from sklearn.cluster import KMeans

# k-meansクラスタリング
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print("cluster", i)
    print(", ".join([countries[k] for k in cluster]))

cluster 0
Syria, Qatar, Israel, Oman, Malta, Lebanon, Iran, Afghanistan, Bahrain, Iraq, Libya, Morocco, Egypt, Jordan, Tunisia
cluster 1
Uzbekistan, Ukraine, Montenegro, Poland, Turkey, Macedonia, Croatia, Belarus, Bulgaria, Albania, Russia, Latvia, Lithuania, Azerbaijan, Armenia, Slovakia, Turkmenistan, Cyprus, Kazakhstan, Slovenia, Estonia, Romania, Serbia, Kyrgyzstan, Moldova, Hungary, Georgia, Tajikistan, Greece
cluster 2
Nigeria, Ghana, Kenya, Malawi, Mauritania, Burundi, Niger, Mozambique, Gambia, Angola, Liberia, Zimbabwe, Sudan, Madagascar, Senegal, Botswana, Guinea, Mali, Zambia, Eritrea, Somalia, Gabon, Rwanda, Uganda, Algeria, Namibia
cluster 3
Bahamas, Dominica, Belize, Samoa, Uruguay, Venezuela, Chile, Cuba, Guyana, Jamaica, Mexico, Nicaragua, Brazil, Peru, Ecuador, Argentina, Suriname, Colombia, Honduras, Fiji
cluster 4
Cambodia, Taiwan, Bangladesh, Thailand, Portugal, Malaysia, Norway, Australia, Liechtenstein, Greenland, France, Tuvalu, Denmark, Iceland, Canada, Switzer

#### 69. t-SNEによる可視化
国名に関する単語ベクトルのベクトル空間をt-SNEで可視化せよ．

t-SNEで単語ベクトルを2次元に圧縮し、散布図で可視化します。

In [ ]:
# !pip install bhtsne

In [ ]:
# import bhtsne

# embedded = bhtsne.tsne(np.array(countries_vec).astype(np.float64), dimensions=2, rand_seed=123)
# plt.figure(figsize=(10, 10))
# plt.scatter(np.array(embedded).T[0], np.array(embedded).T[1])
# for (x, y), name in zip(embedded, countries):
#     plt.annotate(name, (x, y))
# plt.show()